# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

SMALL_SIZE = 26
MEDIUM_SIZE = 30
BIGGER_SIZE = 34
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import os
import math
import scipy
from scipy.integrate import odeint
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
import ast
from time import time
import re
from random import shuffle
import pickle
from tqdm import tqdm_notebook

from importlib import import_module

import dspace
import dspace.plotutils

from utils import int_model, draw_single_pheno_samples, check_oscillation, \
                  process_check_plot_pset, count_parameter_occurence, merge_two_dicts, \
                  random_sample_phenotype, lognuniform, load_model_variables, \
                  build_analyse_design_space, draw_time_course

print 'System information:'
print 'dspace            ' + dspace.__version__
print 'libdesignspace    ' + dspace.__c_toolbox__.__version__
print 'matplotlib        ' + matplotlib.__version__
print 'numpy             ' + np.__version__
print 'sciPy             ' + scipy.__version__

# Bifurcation diagram, phenotype and stability planes for chosen limit cycles
For each element in the following list of lists we draw bifurcation diagrams and phenotype planes. 

Set up lists where the first element is the phenotype casenumber that exists in the index of the df_samples_LC dataframe and where the second element refers to the limit cycle number in the N column. This allows you to choose specific limit cycles to draw the bifurcation diagrams for. 

In [ ]:
my_model = 'Design 4'

### load required variables ###
scipy_model, pset, variables, y0, f, constraints, parbounds, latex_symbols, varnames = load_model_variables(my_model.lower().replace(' ','_')
)

### build design space and save characteristics ###
ds, valid_cases = build_analyse_design_space(my_model, f, constraints, latex_symbols)

In [ ]:
bifurcation_parameter = ['K_zx'] # has to be a list
plane_parameter_combos = [['a_yy','a_zz'],['K_zx', 'g_yy']] # has to be a list of lists with 2 elements
yax_variable = 'x'

LC_to_analyze = [['1906532',1]]

path_to_samples = '../LCs/'+my_model+'/LC_parameters'
df_LC_samples = pd.read_pickle(path_to_samples)  

In [ ]:
SMALL_SIZE = 22
MEDIUM_SIZE = 26
BIGGER_SIZE = 30
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

start_time_total = time()

for phenotype, LC_n in LC_to_analyze:
    print 'Working on phenotype:', phenotype, 'LC number', LC_n
    
    # all LC parameter set belonging to this phenotype
    row = df_LC_samples.loc[[phenotype]].set_index('n').loc[LC_n]
    display(row)

    ########################################
    # make sure figure folders exist
    ########################################
    cwd = os.getcwd()
    dirname = cwd+'../LCs/'+my_model+'/'+phenotype+'/'+str(LC_n)+'/'
    if not os.path.exists(os.path.dirname(dirname)):
        os.makedirs(dirname)

    # image path base
    path_base = '../LCs/'+my_model+'/'+phenotype+'/'+str(LC_n)+'/'

    ########################################
    # save parameters
    ########################################
    row.to_excel(path_base+'parameters.xlsx')

    ########################################
    # setup
    ########################################
    case = ds(phenotype)
    pvals = row['Parameters']
    initial_conditions = row['IC']
    
    ########################################
    # redo limit cycle time course
    ########################################
    # Simulate time course
    oscillates, t, x, initial_conditions, peak_props = check_oscillation(scipy_model,pvals, initial_conditions,ds)
    draw_time_course(t,x,ds,path_base,peak_props,{'s':'Sic1','x':'Clb5','y':'Clb3','z':'Clb2'})

    for current_bifurcation_parameter in bifurcation_parameter:
        ########################################
        # Calculate time courses walking along a 1D line varying the bifurcation parameter
        # observe oscillations by lookin at the max/min of the timecourses
        ########################################
        print 'Bifurcation parameter is currently set to', current_bifurcation_parameter, '=', pvals[current_bifurcation_parameter]

        act_tol_nonlog = list(case.vertices_1D_slice(pvals,current_bifurcation_parameter,log_out=False))
        act_tol = list(case.vertices_1D_slice(pvals,current_bifurcation_parameter,log_out=True))

        # get a sense of how far to change the bifurcation parameter
        print 'phenotype tolerance in NON log10 for the bifurcation parameter:',act_tol_nonlog
        print 'phenotype tolerance in log10 for the bifurcation parameter:',act_tol

        # scan the full model over slightly more than the phenotype range in non-log values 
        scan_start = 0.75*act_tol_nonlog[0][0]; scan_end = 1.25*act_tol_nonlog[1][0]; 

        print 'Scanning over the range',(scan_start,scan_end)
        x_vals1 = np.linspace(scan_start, scan_end, 100) # FORWARD DIRECTION VALUES
        y_vals1 = dict(min_val = [], max_val = []) # init

        # forward integration full model
        # get consistent initial conditions for the first forward case
        # make sure the parameter is set to the first value of x_vals1
        # We remove the values for the auxiliary variables (not defined for the original systems).        
        pvals2 = pvals.copy() # leave original set untouched
        pvals2[current_bifurcation_parameter] = x_vals1[0]
        print 'Update bifurcation parameter to:', current_bifurcation_parameter, '=', pvals2[current_bifurcation_parameter]
        initial_conditions = case.steady_state(pvals2)

        for aux_var in ds(phenotype).auxiliary_variables:
                _=initial_conditions.pop(aux_var)

        start_time = time()
        for x in x_vals1:
            pvals2[current_bifurcation_parameter] = x # update parameter

            # (UN)COMMENTING THIS SWITCHES BETWEEN THE IC APPROACHES 
            initial_conditions = case.steady_state(pvals2) # THIS WILL BASE ALL ICs ON THE SDS PHENOTYPE SS

            y, t, flag = scipy_model(pvals2,initial_conditions,0,5000,100) # initial time course to get to steady-state
            initial_conditions = {i:y[i][-1] for i in y} # use last point as initial condition
            y, t, flag = scipy_model(pvals2,initial_conditions,0,2500,200) # calculate time course from hopefully the steady state to find if there are steady-state oscillations
            y = y[yax_variable] # output concentration on the y-axis
            y_vals1['min_val'].append(np.min(y)) # minimum for oscillations
            y_vals1['max_val'].append(np.max(y)) # maximum for oscillations

        print 'Forward integration took:', round(time() - start_time,2),'seconds.'


        start_time = time()
        ########################################
        # Bifurcation diagram
        ########################################
        fig = plt.figure()
        fig.set_size_inches(15., 7.5)
        ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
        
        ld = {4:{'ls':'--', 'c':'red', 'lw':2}, 
             2:{'ls':'--', 'c':'orange', 'lw':2}, 
             1:{'ls':':', 'c':'black', 'lw':1},
             '1*':{'ls':':', 'c':'black', 'lw':1}, 
             0:{'ls':'-', 'c':'gray', 'lw':2}}

        # Design space eigenvalues: 0,1 or 2 with different line types
        ds.draw_1D_positive_roots(ax, 'log('+yax_variable+')', pvals,
                                  current_bifurcation_parameter,
                                  [0.9*scan_start, 1.1*scan_end],
                                  line_dict=ld
                                  )

        # save the Design space steady state alone
        fig.savefig(path_base+'Design_space_steady_state_'+current_bifurcation_parameter+'.eps', dpi=200, bbox_inches='tight')

        # full model simulations
        ax.fill_between(np.log10(x_vals1), np.log10(y_vals1['min_val']), y2=np.log10(y_vals1['max_val']), color='b', alpha=0.5)
        
        ax.set_xlabel(r'$\log_{10}('+latex_symbols[current_bifurcation_parameter]+')$')
        ax.set_ylabel(r'$\log_{10}('+yax_variable+')$ [a.u.]')
        
        # plot vertical line on the original limit cycle spot
        _=ax.axvline(x=np.log10(pvals[current_bifurcation_parameter]),c='k',ls='--')

        fig.savefig(path_base+'bifurcation_diagram_'+current_bifurcation_parameter+'.eps', dpi=200, bbox_inches='tight')
        print 'Drawing the bifurcation diagram took:', round(time() - start_time,2),'seconds.'


    for combo in plane_parameter_combos:
        start_time = time()
        
        ########################################
        # Phenotype plane
        ########################################
        x_parameter = combo[0]
        y_parameter = combo[1]
        
        # determine bounding box phenotype
        # This gives you the tolerance for the x and y parameters
        # if four corners right, bottom, left, top. But if there are more corners it becomes complex.
        print x_parameter, y_parameter
        vertices = case.vertices_2D_slice(pvals, x_parameter, y_parameter)

        # take the lowest and highest value vertex for each parameter
        xmin = min([el[0] for el in vertices])
        ymin = min([el[1] for el in vertices])
        xmax = max([el[0] for el in vertices])
        ymax = max([el[1] for el in vertices])

        multiplier = 2 # go beyond the boundaries of the phenotype
        range_x = [max(xmin / multiplier,1e-9), min(xmax*multiplier,1e9)]
        range_y = [max(ymin / multiplier,1e-9), min(ymax*multiplier,1e9)]

        fig = plt.figure()
        fig.set_size_inches(15, 7.5)
        ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

        # NOTE: THIS PLOTS LOG-LOG 
        cdict = ds.draw_2D_slice(ax, pvals, x_parameter, y_parameter,
                               range_x, range_y, intersections=[i for i in range(3)])

        # plot the parameter set in the region
        _=ax.plot(np.log10(pvals[x_parameter]), np.log10(pvals[y_parameter]), 'ko', ms=15)

        fig.savefig(path_base+'phenotype_plane_'+x_parameter+'_'+y_parameter+'.eps', dpi=200, bbox_inches='tight')
        print 'Drawing the phenotype plane took:', round(time() - start_time,2),'seconds.'

        start_time = time()
        ########################################
        # Stability plane
        ########################################
        # We define a color dictionary for the stability plots. 
        stability_colors = {'0':(0.0, 1, 1, 1), # green+blue,
                            '0,1':(0, 0, 1, .5), # light blue,
                            '0,1,2':(0.2, 0.2, 0.8, 1.), # bit of everything
                            '1':(0, 0.9, 0., 1.), # light green,
                            '1,2':(1, 0.8, 0., 1.), # RED+green = orange,
                            '2':(1, 1, 0., 1.)} # red+green = yellow


        fig = plt.figure()
        fig.set_size_inches(15, 7.5)
        ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

        # The stability of the fixed points is determined by sampling a grid of nxn points, where n is the value of the Resolution field
        res = 100 # default is 100

        cdict = ds.draw_2D_positive_roots(ax, pvals,
                                        x_parameter, y_parameter,
                                        range_x, range_y,
                                        color_dict=stability_colors,resolution=res,)

        _=ax.plot(np.log10(pvals[x_parameter]), np.log10(pvals[y_parameter]), 'ko', ms=15)

        fig.savefig(path_base+'stability_plane_'+x_parameter+'_'+y_parameter+'.eps', dpi=200, bbox_inches='tight')
        print 'Drawing the stability plane took:', round(time() - start_time,2),'seconds.'

    plt.close('all')

    print 'The whole bifurcation analysis took:', round(time() - start_time_total,2),'seconds.'

# 2D bifurcation diagram

In [ ]:
my_model = 'Design 4'

### load required variables ###
scipy_model, pset, variables, y0, f, constraints, parbounds, latex_symbols, varnames = load_model_variables(my_model.lower().replace(' ','_'))

### build design space and save characteristics ###
ds, valid_cases = build_analyse_design_space(my_model, f, constraints, latex_symbols)

LC_to_analyze = [['1906532',1]]

path_to_samples = '../LCs/'+my_model+'/LC_parameters'
df_LC_samples = pd.read_pickle(path_to_samples)  

# bif_parameters = ['a_yy', 'a_zz'] # has to be a list
bif_parameters = ['g_yy','a_zz']
multipliers = [[0.5, 2],[0.5,2]]
multipliers = [[1./6, 5],[1./3,2]]
yax_variable = 'x'
n = 1500

In [ ]:
SMALL_SIZE = 18
MEDIUM_SIZE = 22
BIGGER_SIZE = 26
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

for phenotype, LC_n in LC_to_analyze:
    print 'Working on phenotype:', phenotype, 'LC number', LC_n
    
    case = ds(phenotype)
    display(case)
    
    # all LC parameter set belonging to this phenotype
    row = df_LC_samples.loc[[phenotype]].set_index('n').loc[LC_n]
#     display(row)

    ########################################
    # make sure figure folders exist
    ########################################
    cwd = os.getcwd()
    dirname = cwd+'../LCs/'+my_model+'/'+phenotype+'/'+str(LC_n)+'/'
    if not os.path.exists(os.path.dirname(dirname)):
        os.makedirs(dirname)
        
    # image path base
    path_base = '../LCs/'+my_model+'/'+phenotype+'/'+str(LC_n)+'/'

    ########################################
    # save parameters
    ########################################
    row.to_excel(path_base+'parameters.xlsx')
    
    ########################################
    # set parameter bounds
    ########################################
    # keep all constant except the two bifurcation parameters 
    pvals = row['Parameters']
    print {pvals[p] for p in bif_parameters}
    
    bounds = [[pvals[p]*multipliers[i][0],pvals[p]*multipliers[i][1]] for i,p in enumerate(bif_parameters)]
    print bounds
    
    # sample n times
    amplitudes = {}
    for i_iter in range(n):
        pvals_sample = pvals.copy()
        
        # get random sample within bounds for each parameter
        for i_bif,p in enumerate(bif_parameters):
            a = np.log10(bounds[i_bif][0])
            b = np.log10(bounds[i_bif][1])
            sample = lognuniform(a,b, 1, base=10)
            
            # update the parameter set
            pvals_sample[p] = sample
        
        par_combo = tuple([pvals_sample[p] for p in bif_parameters])
        
        # calculate time course for this parameter set
        initial_conditions = case.steady_state(pvals_sample) # THIS WILL BASE ALL ICs ON THE SDS PHENOTYPE SS
        oscillates, t, x, initial_conditions, peak_props = check_oscillation(scipy_model, pvals_sample, initial_conditions, ds)

        if oscillates:
            y = x[yax_variable] 
            amplitude = np.max(y) -  np.min(y)
            amplitudes[par_combo] = amplitude
        else:
            amplitudes[par_combo] = 0
        

    ############
    # Visualize results
    ############
    # https://stackoverflow.com/questions/9008370/python-2d-contour-plot-from-3-lists-x-y-and-rho
    x = [el[0] for el in amplitudes]
    y = [el[1] for el in amplitudes]
    z = [amplitudes[el] for el in amplitudes]

    import scipy.interpolate
    from mpl_toolkits.axes_grid1 import make_axes_locatable

    fig = plt.figure()
    fig.set_size_inches(10, 7)
    ax = plt.gca()

    # Set up a regular grid of interpolation points
    xi, yi = np.linspace(min(x), max(x), 50), np.linspace(min(y), max(y), 50)
    xi, yi = np.meshgrid(xi, yi)

    # Interpolate
    rbf = scipy.interpolate.Rbf(x, y, z, function='linear')
    zi = rbf(xi, yi)

    im = plt.imshow(zi, vmin=min(z), vmax=max(z), origin='lower',cmap='Greens', 
                    extent=[min(x), max(x), min(y), max(y)], aspect='auto')
    
#     plt.scatter(x, y, c=z, cmap='cool', alpha = 0.5) # to see if we sample densely enough

    plt.xlabel(r'$'+latex_symbols[bif_parameters[0]]+'$')
    plt.ylabel(r'$'+latex_symbols[bif_parameters[1]]+'$')
    plt.plot(pvals[bif_parameters[0]], pvals[bif_parameters[1]],'ro',ms=15)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im,cax=cax)

    plt.show()

    fig.savefig(path_base+'2D_bifurcation_diagram_'+bif_parameters[0]+'_'+bif_parameters[1]+'.eps', dpi=300, bbox_inches='tight')